In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

I. Phân tích bài toán:
- Nhiệm vụ phải làm trong bài toán là **phân loại** các câu **insincere** và **sincere** có trên hệ thống của Quora
- Tập đầu vào là tập các câu hỏi tiếng anh được cho dưới dạng *text* và đi kèm là *id* của từng câu cũng như *nhãn dán (label)* cho từng câu (**insincere** là 1 và **sincere** là 0) đầu ra là các giá trị 0 hoặc 1 là nhãn mà mô hình đoán
- Với bài toán trên, ta cần chuyển dữ liệu từ dạng *text* sang ma trận đặc trưng dưới dạng *số* sau đó kết hợp với mô hình học máy (mô hình Logistic Regression) để huấn luyện và dự đoán kết quả đầu ra 

In [ ]:
# Đọc dữ liệu từ bài toán: 
df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')

In [ ]:
# Lấy ra 1 số các câu trong tập input
df.head(5)

TỔNG QUÁT VỀ TẬP INPUT:
- Tổng cộng 1306122 câu hỏi dưới dạng text được gán nhãn và không có giá trị null
- Target (nhãn) là tập số nguyên gồm 2 giá trị 1 và 0
- Tổng số câu **sincere** là 1225312 số câu **insincere** là 80810

In [ ]:
# Thông tin về tập input:
df.info()

In [ ]:
import matplotlib.pyplot as plt

# Mô phỏng về độ tương quan giữa các câu insincere và sincere dưới dạng biểu đồ:

val = df.target.value_counts().values
names = ['Sincere', 'Insincere']
plt.bar(names, val)
plt.suptitle('Number of Sincere and Insincere Questions')
plt.show()

In [ ]:
val

II. Tiền xử lý cho dữ liệu đầu vào:
- Với dữ liệu dạng text, ta cần chuyển sang số và biểu diễn thông qua các ma trận để huấn luyện mô hình học máy
- Các bước tiền xử lý trong bài gồm có:
    - Loại bỏ các dấu câu, và các từ stop_words có trong câu hỏi (các từ stop_words thường là những từ viết tắt và không mang nhiều ý nghĩa và có thể ảnh hưởng đến trọng số của những từ quan trọng trong câu)
    - Loại bỏ các số có trong câu (cũng giống như stop_words các số thường không mang nhiều ý nghĩa trong việc phân loại câu nên cần loại bỏ)
    - Sử dụng phương pháp **TF_IDF** hoặc đếm số lần xuất hiện của những từ đặc trưng trong câu để biểu diễn ma trận trọng số của những từ quan trọng trong câu hỏi (trong bài này ta sẽ kiểm tra cả 2 cách và so sánh kết quả giữa chúng)
- Sau đó, ta sẽ chia tập input thành 2 phần: 1 phần để huấn luyện và 1 phần để kiểm tra độ chính xác của mô hình (tỉ lệ là 7:3)
    
    
*** Phương pháp TF_IDF**:
- Là phương pháp cơ bản trong việc xử lý dữ liệu văn bản trong xử lý ngôn ngữ tự nhiên.
- Mục đích: Cân bằng mức độ quan trọng giữa các từ và loại bỏ 1 số những từ không cần thiết bằng cách tính trọng số TF (Term frequence) và IDF (Inverse document frequence)
- TF (Term frequence) được tính như sau:
    **TF(t, d) = ( số lần từ t xuất hiện trong văn bản d) / (tổng số từ trong văn bản d)** ========> Tần suất của từ t (trong khoảng [0,1])
- IDF (Inverse document frequence) được tính như sau:
    **IDF(t, D) = log_e( Tổng số văn bản trong tập mẫu D/ Số văn bản có chứa từ t )** ===========> Mức độ quan trọng của từ t
- Trọng số TF_IDF của từ t là ***TF(t, d) * IDF(t, D)***


In [ ]:
from string import punctuation, digits
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()
# Khai báo tập các dấu câu và tập các stop_words của tiếng Anh
puncs = set(punctuation)
stop_w = set(stopwords.words('english'))

In [ ]:
print(puncs)
print(stop_w)

In [ ]:
# Tiền xử lý dữ liệu:
def lemSentence(sentence): # Xây dựng lại cấu trúc câu sau khi đã loại bỏ các ký tự và các từ không cần thiết
    token_words = word_tokenize(sentence)
    lem_sentence = []
    for word in token_words:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        lem_sentence.append(" ")
    return "".join(lem_sentence)

def clean(text): # Loại bỏ các ký tự không cần thiết 
    # Loại bỏ các dấu câu
    text = text.translate(str.maketrans('', '', punctuation))
    # Loại bỏ các số
    text = text.translate(str.maketrans('', '', digits))
    # Loại bỏ các stop_words
    text = [w for w in word_tokenize(text) if not w.lower() in stop_w]
    text = ' '.join(text)
    # Thiết lập lại cấu trúc của câu
    text = lemSentence(text)
    return text

In [ ]:
# Lấy ra tập train và tập nhãn dán của dữ liệu
train_quests = df.question_text
train_labels = df.target

In [ ]:
# Thực hiện việc tiền xử lý với dữ liệu đầu vào và lưu kết quả vào 1 hàng mới có tên là "question_text_cleaned"
df['question_text_cleaned'] = df.question_text.apply(lambda x: clean(x))

In [ ]:
# Lấy ra 1 số câu sau khi đã thực hiện tiền xử lý
df.head(5)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split

* Sau khi thực hiện việc loại bỏ các ký tự và các từ ngữ không cần thiết, ta sẽ thực hiện việc mã hóa TF_IDF cho tập từ vựng mới 
* Thư viện sklearn đã hỗ trợ việc tính toán cũng như chuyển đổi mà trận TF_IDF (tính toán trọng số và chuyển đổi từ dạng text sang ma trận trọng số) tương tự với cách đếm số từ quan trọng trong 1 câu hỏi
*Link: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html ----------------
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
# Khai báo hàm thực hiện tính trọng số
count_vectorizer = CountVectorizer()
tf_idf_vectorizer = TfidfVectorizer()
# Chia tập dữ liệu đầu vào thành 2 phần (tập train và test theo tỷ lệ (7:3))
x_train, x_test, y_train, y_test = train_test_split(df['question_text_cleaned'], train_labels, test_size=0.3)
# Tiến hành tính toán trọng số của các từ trong tập huấn luyện
count_vectorizer.fit(x_train)
tf_idf_vectorizer.fit(x_train)
# Biến đổi các câu trong tập train thành ma trận trọng số
vt_count_train = count_vectorizer.transform(x_train)
vt_count_test = count_vectorizer.transform(x_test)

tfidf_vt_train = tf_idf_vectorizer.transform(x_train)
tfidf_vt_test = tf_idf_vectorizer.transform(x_test)

In [ ]:
# Ma trận trọng số của các câu hỏi sẽ là input cho mô hình và có shape là (914285, 176283)
# trong đó: 914285 là số lượng các câu hỏi 
#           176283 là kích thước mà trận trọng số dại diện cho câu hỏi đó được xây dựng từ tập từ vựng của dữ liệu đầu vào 
tfidf_vt_train.shape

III. Huấn luyện mô hình và báo cáo kết quả:
* Sử dụng mô hình học máy Logistics Regression để thực thiện huấn luyện và kiểm tra độ chính xác

In [ ]:
# Khái báo mô hình
model_1 = LogisticRegression(n_jobs=10, solver='saga', C=0.1, verbose=1)
model_2 = LogisticRegression(n_jobs=10, solver='sag', C=0.1, verbose=1)
# Tiến hành huấn luyện trên tập dữ liệu đã được mã hóa bằng TF_IDF
model_1.fit(vt_count_train, y_train)
model_2.fit(tfidf_vt_train, y_train)

In [ ]:
# Độ chính xác trên tập train
preds = model_1.predict(vt_count_train)
accuracy = accuracy_score(y_train, preds)
print("Accuracy in train set: ", accuracy)
# Dự đoán bằng mô hình vừa huấn luyện
y_preds = model_1.predict(vt_count_test)
# Ma trận lỗi của mô hình
print(f"Confusion matrix: ") 
print(confusion_matrix(y_test, y_preds))

In [ ]:
# Ma trận lỗi của mô hình:
plot_confusion_matrix(model_1, vt_count_test, y_test)
plot_confusion_matrix(model_2, tfidf_vt_test, y_test)
plt.show()  

* Từ 2 ma trận lỗi trên, ta có thể thấy việc mã hóa dữ liệu đầu vào bằng cách đếm số lần xuất hiện của các từ trong câu cho độ chính xác cao hơn việc mã hóa bằng phương pháp TF_IDF

In [ ]:
# Báo cáo kết quả thực hiện được 
print("Classificaiton report:\n", classification_report(y_test, y_preds, target_names=["Sincere", "Insincere"]))